In [ ]:
import os
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
df=pd.read_csv(r'.\data.\unprocessed_EPS2.csv',index_col=0)
df = df[df['Validity'] != 0]
df

In [ ]:
from pytorch_tabnet.tab_model import TabNetRegressor,TabNetClassifier

import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error,accuracy_score

import pandas as pd
import numpy as np



import os
import wget
from pathlib import Path

In [ ]:
df1 = df.drop(['D1', 'D0', 'Validity', 'ipk', 'irms', 'Vo', 'nZVS'], axis=1)
df1 = df1.reset_index(drop=True)
df1

In [ ]:
target = 'Ptotal'
if "Set" not in df1.columns:
    df1["Set"] = np.random.choice(["train", "valid", "test"], p =[.8, .1, .1], size=(df1.shape[0],))

train_indices = df1[df1.Set=="train"].index
valid_indices = df1[df1.Set=="valid"].index
test_indices = df1[df1.Set=="test"].index

In [ ]:
clf = TabNetRegressor(n_d=16,n_a=16)

In [ ]:
unused_feat = ['Set']
features = [ col for col in df1.columns if col not in unused_feat+[target]] 

In [ ]:
X_train = df1[features].values[train_indices]
y_train = df1[target].values[train_indices].reshape(-1, 1)

X_valid = df1[features].values[valid_indices]
y_valid = df1[target].values[valid_indices].reshape(-1, 1)

X_test = df1[features].values[test_indices]
y_test = df1[target].values[test_indices].reshape(-1, 1)

In [ ]:
max_epochs = 800 if not os.getenv("CI", False) else 2

In [ ]:
clf.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['mse'],
    max_epochs=max_epochs,
    patience=300,
    batch_size=1024, virtual_batch_size=128,
    num_workers=0,
    #augmentations=aug
) 

In [ ]:
preds = clf.predict(X_test)

y_true = y_test

test_score = mean_squared_error(y_pred=preds, y_true=y_true)

In [ ]:
test_score 

In [ ]:
df2 = df.drop(['D1', 'D0', 'Validity', 'ipk', 'irms', 'Vo', 'Ptotal'], axis=1)
df2 = df2.reset_index(drop=True)

In [ ]:
df2

In [ ]:
target = 'nZVS'
if "Set" not in df2.columns:
    df2["Set"] = np.random.choice(["train", "valid", "test"], p =[.8, .1, .1], size=(df2.shape[0],))
train_indices = df2[df2.Set=="train"].index
valid_indices = df2[df2.Set=="valid"].index
test_indices = df2[df2.Set=="test"].index

In [ ]:
clf2 = TabNetClassifier(n_d=4,n_a=4)

In [ ]:
unused_feat1 = ['Set']
features1 = [ col for col in df2.columns if col not in unused_feat1+[target]] 

In [ ]:
features1

In [ ]:
X_train = df2[features1].values[train_indices]
y_train = df2[target].values[train_indices]

X_valid = df2[features1].values[valid_indices]
y_valid = df2[target].values[valid_indices]

X_test = df2[features1].values[test_indices]
y_test = df2[target].values[test_indices]

In [ ]:
max_epochs =110 if not os.getenv("CI", False) else 2

In [ ]:
clf2.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['accuracy'],
    max_epochs=max_epochs,
    patience=80,
    batch_size=4096, virtual_batch_size=128,
    num_workers=0,
    #augmentations=aug
) 

In [ ]:
preds_mapper = { idx : class_name for idx, class_name in enumerate(clf2.classes_)}

preds = clf2.predict_proba(X_test)

y_pred = np.vectorize(preds_mapper.get)(np.argmax(preds, axis=1))

test_acc = accuracy_score(y_pred=y_pred, y_true=y_test)

In [ ]:
test_acc